This file creates prompts for use with LLaMA and saves them in Parquet format.

In [ ]:
from datasets import load_dataset
import pandas as pd

rootdir = "data/parquet/llama_original/"
saved_dir = "data/parquet/fixed/"

In [ ]:
import os
import json
from typing import List, Dict
from math500.math_utils import list_fewshot_samples

def generate_prompt(few_shot_examples: List[Dict], query: str) -> str:
    start_prompt = "<|start_header_id|>user<|end_header_id|>\n\nSolve the following math problem efficiently and clearly:\n\n- For simple problems (2 steps or fewer):\nProvide a concise solution with minimal explanation.\n\n- For complex problems (3 steps or more):\nUse this step-by-step format:\n\n## Step 1: [Concise description]\n[Brief explanation and calculations]\n\n## Step 2: [Concise description]\n[Brief explanation and calculations]\n\n...Regardless of the approach, always conclude with:\n\nTherefore, the final answer is: $\\boxed{answer}$. I hope it is correct.\n\nWhere [answer] is just the final number or expression that solves the problem.\n\n"
    prompt = ""
    
    # for example in few_shot_examples:
    #     prompt += start_prompt + "Problem: " + example["problem"] + "<|eot_id|><|start_header_id|>assistant<|end_header_id|>" + "\n\n" 
    #     prompt += example["solution"] + "<|eot_id|>"
    
    prompt += start_prompt +  "Problem: " + query + "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
    
    return prompt


file_path = f"data/math500/test.jsonl"
with open(file_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

all_records = []

for idx, entry in (enumerate(dataset)): 
    question_id = idx 
    question = entry['problem']
    answer = entry['solution']
    prompt = generate_prompt(list_fewshot_samples(), question)

    record = {
            "question_id": question_id,
            "input_question": question,
            "input_correct_responses": answer,
            "input_final_prompts": [prompt],
        }
    all_records.append(record)

df_all = pd.DataFrame(all_records)
df_all.to_parquet("data/parquet/fixed/math500_0shot.parquet", index=False)
print(prompt)


<|start_header_id|>user<|end_header_id|>

Solve the following math problem efficiently and clearly:

- For simple problems (2 steps or fewer):
Provide a concise solution with minimal explanation.

- For complex problems (3 steps or more):
Use this step-by-step format:

## Step 1: [Concise description]
[Brief explanation and calculations]

## Step 2: [Concise description]
[Brief explanation and calculations]

...Regardless of the approach, always conclude with:

Therefore, the final answer is: $\boxed{answer}$. I hope it is correct.

Where [answer] is just the final number or expression that solves the problem.

Problem: Altitudes $\overline{AD}$ and $\overline{BE}$ of $\triangle ABC$ intersect at $H$.  If $\angle BAC = 54^\circ$ and $\angle ABC = 52^\circ$, then what is $\angle AHB$?<|eot_id|><|start_header_id|>assistant<|end_header_id|>



In [ ]:
def format_example_with_content(start_prompt, question, options, cot_content="", end_prompt=""):
    text = "A: Let's think step by step. "
    if cot_content.startswith("A: "):
        cot_content = cot_content[len(text):]
    example = start_prompt+ "Question: {}\n".format(question)
    choice_map = "ABCDEFGHIJ"
    
    for i in range(len(choice_map)):
        if i < len(options):
            example += f"{choice_map[i]}. {options[i]}\n"
        else:
            example += f"{choice_map[i]}. N/A\n"
    example += end_prompt
    
    example += "\n\n" + cot_content + "<|eot_id|>"
    return example

def preprocess(test_df):
    res_df = []
    for each in test_df:
        options = [opt for opt in each["options"] if opt != "N/A"]
        each["options"] = options
        res_df.append(each)
    res = {}
    for each in res_df:
        if each["category"] not in res:
            res[each["category"]] = []
        res[each["category"]].append(each)
    return res

def load_mmlu_pro():
    dataset = load_dataset("TIGER-Lab/MMLU-Pro")
    test_df, val_df = dataset["test"], dataset["validation"]
    test_df = preprocess(test_df)
    val_df = preprocess(val_df)
    return test_df, val_df

import re

def remove_paren_after_answer(cot_content: str) -> str:
    pattern = re.compile(r'(The answer is )\(([^)]+)\)(\.)?')
    return pattern.sub(r'The best answer is \2\3', cot_content)

def format_example(question, options, end_prompt):
    example = "Question: {}\n".format(question)
    choice_map = "ABCDEFGHIJ"
    for i, opt in enumerate(options):
        if i < len(options):
            example += f"{choice_map[i]}. {options[i]}\n"
        else:
            example += f"{choice_map[i]}. N/A\n"
    example += end_prompt
    return example


In [ ]:
import random

dataset, dev_df = load_mmlu_pro()

start_prompt = "<|start_header_id|>user<|end_header_id|>\n\nGiven the following question and candidate answers, choose the best answer.\n"
end_prompt = "\nYour response should end with \"The best answer is [the_answer_letter].\" where the [the_answer_letter] is a letter from the provided choices.\n\nLet's think step by step.<|eot_id|><|start_header_id|>assistant<|end_header_id>"

all_records = []
all_commented_records = []

subjects = list(dataset.keys())
for subject in subjects:
    random.seed(42)
    test_data = random.sample(dataset[subject], min(300, len(dataset[subject])))
    
    few_shot_prompt = ""
    for each in dev_df[subject]:
        cot_content = remove_paren_after_answer(each["cot_content"])
        few_shot_prompt += format_example_with_content(start_prompt, 
                                                       each["question"], 
                                                       each["options"], 
                                                       cot_content, 
                                                       end_prompt)
    
    for entry in test_data:
        question_id = entry['question_id']
        category = entry['category']
        input_question = entry['question']
        input_choice_list = entry['options']
        input_correct_responses = entry['answer']
        
        input_final_prompt = (few_shot_prompt +
                              start_prompt +
                              format_example(entry['question'], entry['options'], end_prompt) +
                              "\n\n")
        
        commented_prompt = start_prompt + format_example(entry['question'], entry['options'], end_prompt) + "\n\n"
        
        record = {
            "question_id": question_id,
            "category": category,
            "input_question": input_question,
            "input_correct_responses": [input_correct_responses],
            "input_choice_list": input_choice_list,
            "input_final_prompts": [input_final_prompt],
        }
        all_records.append(record)
        
        commented_record = {
            "question_id": question_id,
            "category": category,
            "input_question": input_question,
            "input_correct_responses": input_correct_responses,
            "input_choice_list": input_choice_list,
            "input_final_prompts": [commented_prompt]
        }
        all_commented_records.append(commented_record)

df_all = pd.DataFrame(all_records)
df_commented = pd.DataFrame(all_commented_records)

df_commented.to_parquet("mmlu_prompts_0shot_gpt.parquet", index=False)


In [68]:
print(df_all.iloc[0]['input_final_prompts'][0])

<|start_header_id|>user<|end_header_id|>

Given the following question and candidate answers, choose the best answer.
Question: In contrast to _______, _______ aim to reward favourable behaviour by companies. The success of such campaigns have been heightened through the use of ___________, which allow campaigns to facilitate the company in achieving _________ .
A. Boycotts, Buyalls, Blockchain technology, Increased Sales
B. Buycotts, Boycotts, Digital technology, Decreased Sales
C. Boycotts, Buycotts, Digital technology, Decreased Sales
D. Buycotts, Boycotts, Blockchain technology, Charitable donations
E. Boycotts, Buyalls, Blockchain technology, Charitable donations
F. Boycotts, Buycotts, Digital technology, Increased Sales
G. Buycotts, Boycotts, Digital technology, Increased Sales
H. Boycotts, Buycotts, Physical technology, Increased Sales
I. Buycotts, Buyalls, Blockchain technology, Charitable donations
J. Boycotts, Buycotts, Blockchain technology, Decreased Sales

Your response sh

In [94]:
from collections import namedtuple
from typing import Dict, List, Tuple, Union
import random

Example = namedtuple('Example', ['question', 'choice1', 'choice2', 'choice3', 'choice4', 'correct_index'])
LETTER_TO_INDEX = {'A': 0, 'B': 1, 'C': 2, 'D': 3}


def generate_prompt_from_examples(json_data, with_explanations=True):
    output = ""
    for q in json_data["questions"]:
        output += "<|start_header_id|>user<|end_header_id|>\n" 
        output += f"Here are some example questions from experts. An explanation is given before the final answer. Answer the final question yourself, giving your reasoning beforehand.\n"
        output += f'Question: {q["question"]}\nChoices:\n'
        for choice, value in q["choices"].items():
            output += f'({choice}) {value}\n'
        output += "\nGive step by step reasoning before you answer, and when you're ready to answer, please use the format \"The correct answer is (insert answer here)\":\n"

        if with_explanations:
            output += f"Let's think step by step:<|eot_id|><|start_header_id|>assistant<|end_header_id|> \n{q['explanation']}\n"
        
        output += f'The correct answer is ({q["correct_answer"]})<|eot_id|>\n'
    
    return output

def chain_of_thought_prompt(json_data, example: Example) -> str:
    """Creates a chain-of-thought prompt given a single example."""
    prompt = generate_prompt_from_examples(json_data, with_explanations=True)
    prompt += "<|start_header_id|>user<|end_header_id|>\n" 
    prompt += f"Here are some example questions from experts. An explanation is given before the final answer. Answer the final question yourself, giving your reasoning beforehand.\n"
    prompt += f"Question: {example.question}"
    prompt += f"\nChoices:\n(A) {example.choice1}\n(B) {example.choice2}\n(C) {example.choice3}\n(D) {example.choice4}"
    prompt += "\nGive step by step reasoning before you answer, and when you're ready to answer, please use the format \"The correct answer is (insert answer here)\":\n"
    prompt += "Let's think step by step:"
    return prompt


def zero_shot_chain_of_thought_prompt(example: Example) -> str:
    prompt = f"What is the correct answer to this question: {example.question}"
    prompt += f"\n\nChoices:\n(A) {example.choice1}\n(B) {example.choice2}\n(C) {example.choice3}\n(D) {example.choice4}"
    prompt += "\nGive step by step reasoning before you answer, and when you're ready to answer, please use the format \"The correct answer is (insert answer here)\":\n"
    prompt += "Let's think step by step:"
    return prompt


def load_examples(path: str, seed: int) -> List[Example]:
    """Load questions from csv file and return a list of Example namedtuples."""
    question_df = pd.read_csv(path)
    random.seed(seed)

    def shuffle_choices_and_create_example(row) -> Example:
        list_choices = [row['Incorrect Answer 1'], row['Incorrect Answer 2'], row['Incorrect Answer 3'], row['Correct Answer']]
        random.shuffle(list_choices)
        example = Example(row.Question, list_choices[0], list_choices[1], list_choices[2], list_choices[3],
                          list_choices.index(row['Correct Answer']))
        return example

    return [shuffle_choices_and_create_example(row) for _, row in question_df.iterrows()]

In [95]:
import json 

examples = load_examples("data/gpqa/gpqa_diamond.csv", seed=0)
with open("gpqa/chain_of_thought_examples.json", 'r') as f:
    fewshot = json.load(f)


In [ ]:
rows_0shot = []
rows_fewshot = []

system_prompt = "<|start_header_id|>system<|end_header_id|>\n" 
system_prompt += "You are a very intelligent assistant, who follows instructions directly.<|eot_id|>\n\n"
start_prompt = "<|start_header_id|>user<|end_header_id|>\n" 
end_prompt = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

for example in examples:

    prompt_0shot = system_prompt + zero_shot_chain_of_thought_prompt(example) + end_prompt


    prompt_fewshot = system_prompt + chain_of_thought_prompt(fewshot, example) + end_prompt
    

    choice_list = [example.choice1, example.choice2, example.choice3, example.choice4]
    correct_answer_text = ["A", "B", "C", "D"][example.correct_index]


    rows_0shot.append({
        "input_question": example.question,
        "input_correct_responses": [correct_answer_text],  
        "input_final_prompts": [prompt_0shot],
        "is_correct": None,  
        "input_question_hash": hash(example.question),
        "input_choice_list": choice_list,
        "output_prediction_text": ""
    })


    rows_fewshot.append({
        "input_question": example.question,
        "input_correct_responses": [correct_answer_text],
        "input_final_prompts": [prompt_fewshot],
        "is_correct": None,
        "input_question_hash": hash(example.question),
        "input_choice_list": choice_list,
        "output_prediction_text": ""
    })


df_0shot = pd.DataFrame(rows_0shot)
df_fewshot = pd.DataFrame(rows_fewshot)


df_0shot.to_parquet(saved_dir + "gpqa_0shot.parquet")
df_fewshot.to_parquet(saved_dir + "gpqa_fewshot.parquet")

print("Saved")

Saved


In [ ]:
print(rows_fewshot[0]['input_final_prompts'][0])

<|start_header_id|>system<|end_header_id|>
You are a very intelligent assistant, who follows instructions directly.<|eot_id|>

<|start_header_id|>user<|end_header_id|>
Here are some example questions from experts. An explanation is given before the final answer. Answer the final question yourself, giving your reasoning beforehand.
Question: In a given population, 1 out of every 400 people has a cancer caused by a completely recessive allele, b. Assuming the population is in Hardy-Weinberg equilibrium, which of the following is the expected proportion of individuals who carry the b allele but are not expected to develop the cancer?
Choices:
(A) 1/400
(B) 19/400
(C) 20/400
(D) 38/400

Give step by step reasoning before you answer, and when you're ready to answer, please use the format "The correct answer is (insert answer here)":
Let's think step by step:<|eot_id|><|start_header_id|>assistant<|end_header_id|> 
The expected proportion of individuals who carry the b allele but are not expe

In [98]:
print(rows_0shot[0]['input_final_prompts'][0])

<|start_header_id|>system<|end_header_id|>
You are a very intelligent assistant, who follows instructions directly.<|eot_id|>

What is the correct answer to this question: Two quantum states with energies E1 and E2 have a lifetime of 10^-9 sec and 10^-8 sec, respectively. We want to clearly distinguish these two energy levels. Which one of the following options could be their energy difference so that they can be clearly resolved?


Choices:
(A) 10^-9 eV
(B) 10^-11 eV
(C) 10^-8 eV

(D) 10^-4 eV
Give step by step reasoning before you answer, and when you're ready to answer, please use the format "The correct answer is (insert answer here)":
Let's think step by step:<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [57]:
start_prompt = "<|start_header_id|>user<|end_header_id|>\n" 

end_prompt = "\n\nEnd your answer with \"Answer <answer>\". Think step by step.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n" 

In [ ]:
def create_final_prompt(question, fewshot):
    input_final_prompt = ""
    for qa in fewshot:
        input_final_prompt += f"{start_prompt}Q: {qa['Q']}{end_prompt}A: {qa['A']}<|eot_id|>\n"
    input_final_prompt += f"{start_prompt}Q: {question}{end_prompt}"
    return [input_final_prompt]

dataset = json.load(open(f'data/hotpotqa/hotpotqa.json'))

df = pd.DataFrame(dataset)

parquet_file = "data/hotpotqa/hotpotqa.parquet"
df.to_parquet(parquet_file, engine="pyarrow", index=False)

with open("hotpotqa/react_prompt.json", 'r') as f:
    fewshot = json.load(f)

df = pd.DataFrame(dataset)


processed_data = []
for entry in dataset:
    input_final_prompt = create_final_prompt(entry['question'], fewshot)
    processed_data.append(input_final_prompt)

df['input_final_prompts'] = processed_data
print(df.iloc[0]['input_final_prompts'][0])

parquet_file = saved_dir + "hotpotqa_fewshot.parquet"
df.to_parquet(parquet_file, engine="pyarrow", index=False)

<|start_header_id|>user<|end_header_id|>
Q: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

End your answer with "Answer <answer>". Think step by step.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

A: Let’s think step by step.  The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft. Answer 1,800 to 7,000 ft<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Q: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

End your answer with "Answer <answer>". Think step by step.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

A: Let’s think step by step. Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon. Answer Richard Nixon<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Q: Which documentary is abou

In [ ]:
dataset = json.load(open(f'data/hotpotqa/hotpotqa.json'))

df = pd.DataFrame(dataset)

parquet_file = "data/hotpotqa/hotpotqa-bm25.parquet"
df.to_parquet(parquet_file, engine="pyarrow", index=False)

with open("hotpotqa/react_prompt.json", 'r') as f:
    fewshot = json.load(f)

processed_data = []
for entry in dataset:
    input_final_prompt = start_prompt + entry['question'] + end_prompt
    processed_data.append([input_final_prompt])

df['input_final_prompts'] = processed_data
print(df.iloc[0]['input_final_prompts'][0])
parquet_file = saved_dir + "hotpotqa_0shot.parquet"
df.to_parquet(parquet_file, engine="pyarrow", index=False)

<|start_header_id|>user<|end_header_id|>
What is the nationality of the foreign born victim of Singapore's caning punishment before Oliver Fricker experienced the same?

End your answer with "Answer <answer>". Think step by step.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [ ]:
# https://huggingface.co/datasets/EleutherAI/drop
dataset = pd.read_parquet("data/drop/drop_sub.parquet", engine="pyarrow")

df = pd.DataFrame(dataset)

with open("drop/prompt.json", 'r') as f:
    fewshot = json.load(f)

end_prompt = "\n\nEnd your answer with \"So the answer is <answer>\". Think step by step.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"  

def create_final_prompt(entry, fewshot):
    input_final_prompt = ""
    for qa in fewshot:
        input_final_prompt += f"{start_prompt}Q: {qa['Q']}{end_prompt}A: {qa['A']}<|eot_id|>\n"
    input_final_prompt += f"{start_prompt}Q: {entry['passage']}{entry['question']}{end_prompt}"
    return [input_final_prompt]


processed_data = []
for _, entry in dataset.iterrows():  
    input_final_prompt = create_final_prompt(entry, fewshot)
    processed_data.append(input_final_prompt)

df['input_final_prompts'] = processed_data
print(df.iloc[0]['input_final_prompts'][0])

parquet_file = saved_dir + "drop_fewshot.parquet"
df.to_parquet(parquet_file, engine="pyarrow", index=False)

<|start_header_id|>user<|end_header_id|>
Q: Since the 1970s, U.S. governments have negotiated managed-trade agreements, such as the North American Free Trade Agreement in the 1990s, the Dominican Republic-Central America Free Trade Agreement in 2006, and a number of bilateral agreements. In Europe, six countries formed the European Coal and Steel Community in 1951 which became the European Economic Community in 1958. Two core objectives of the EEC were the development of a common market, subsequently renamed the single market, and establishing a customs union between its member states. How many years did the European Coal and Steel Community exist?

End your answer with "So the answer is <answer>". Think step by step.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

A: According to the passage, the European Coal and Steel Community was established in 1951 and became the EEC in 1958. 1958 - 1951 = 7. So the answer is 7.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Q: In the

In [ ]:
dataset = pd.read_parquet("data/drop/drop_sub.parquet", engine="pyarrow")

df = pd.DataFrame(dataset)

with open("drop/prompt.json", 'r') as f:
    fewshot = json.load(f)
    

processed_data = []
for _, entry in dataset.iterrows():  
    input_final_prompt = start_prompt + entry['passage'] + entry['question'] + end_prompt
    processed_data.append([input_final_prompt])

df['input_final_prompts'] = processed_data
print(df.iloc[0]['input_final_prompts'][0])
parquet_file = saved_dir + "drop_0shot.parquet"
df.to_parquet(parquet_file, engine="pyarrow", index=False)

<|start_header_id|>user<|end_header_id|>
As of the census of 2000, there were 49,129 people, 18,878 households, and 13,629 families residing in the county.  The population density was 88 people per square mile (34/km2).  There were 21,779 housing units at an average density of 39 per square mile (15/km2).  The racial makeup of the county was 74.4% Race (United States Census), 20.4% Race (United States Census) or Race (United States Census), 0.60% Race (United States Census), 1.1% Race (United States Census), 0.15% Race (United States Census), 1.3% from Race (United States Census), and 2.2% from two or more races.  3.4% of the population were Race (United States Census) or Race (United States Census) of any race. 2.85% of the population reported speaking Spanish language at home, while 1.51% speak German language.How many more housing units are there than households?

End your answer with "So the answer is <answer>". Think step by step.<|eot_id|><|start_header_id|>assistant<|end_header_

In [ ]:
'''
MuSR Llama
'''
from musr.musr import MuSRDataset

from musr.op_icl_fixed import op_fewshot, few_shot_op_instruction, test_op_instruction
from musr.ta_icl_fixed import ta_fewshot, few_shot_ta_instruction, test_ta_instruction


saved_dir = "data/parquet/fixed/"
os.makedirs(saved_dir, exist_ok=True)

task = 'musr_location'

if task == "musr_location":
    op_path = 'data/musr/object_placements.json'
    dataset = MuSRDataset(op_path)
    few_shot_examples = op_fewshot  
    few_instruction = few_shot_op_instruction
    test_instruction = test_op_instruction
elif task == 'musr_efficiently':
    ta_path = 'data/musr/team_allocation.json'
    dataset = MuSRDataset(ta_path)
    few_shot_examples = ta_fewshot
    few_instruction = few_shot_ta_instruction
    test_instruction = test_ta_instruction


few_shot_data = []
zero_shot_data = []


for entry in dataset:

    question = entry['question'].strip()
    answers = [entry['answerKey']]  
    context = entry['context'].strip()
    choices = entry['choices']['text']

    labels = ['A', 'B', 'C', 'D', 'E', 'F'][:len(choices)]
    choice_str = '\n'.join([f'{labels[idx]}: {choices[idx]}' for idx in range(len(choices))])
    original_question_part = f"{context}\n\n{question}\n\n{choice_str}"
    system_prompt = entry['prompt_parts']['cot_system_prompt']
    

    final_prompt_few = ""

    for (q, a) in few_shot_examples:

        final_prompt_few += "<|start_header_id|>user<|end_header_id|>\n"
        final_prompt_few += system_prompt + "\n\n" + q + "\n\n" + few_instruction + "\n"
        final_prompt_few += "<|eot_id|>\n"

        final_prompt_few += "<|start_header_id|>assistant<|end_header_id|>\n"
        final_prompt_few += a + "\n"
        final_prompt_few += "<|eot_id|>\n"
        

    final_prompt_few += "<|start_header_id|>user<|end_header_id|>\n"
    final_prompt_few += system_prompt + "\n\n" + original_question_part + "\n\n" + test_instruction + "\n"
    final_prompt_few += "<|eot_id|>\n"

    final_prompt_few += "<|start_header_id|>assistant<|end_header_id|>\n"
    

    final_prompt_zero = ""
    final_prompt_zero += "<|start_header_id|>user<|end_header_id|>\n"
    final_prompt_zero += system_prompt + "\n\n" + original_question_part + "\n\n" + test_instruction + "\n"
    final_prompt_zero += "<|eot_id|>\n"
    final_prompt_zero += "<|start_header_id|>assistant<|end_header_id|>\n"
    
    few_shot_data.append({
         "question": question,
         "answers": answers,
         "input_final_prompts": [final_prompt_few],
         "entry": entry
    })
    
    zero_shot_data.append({
         "question": question,
         "answers": answers,
         "input_final_prompts": [final_prompt_zero],
         "entry": entry
    })


df_few = pd.DataFrame(few_shot_data, columns=["question", "answers", "input_final_prompts"])
df_zero = pd.DataFrame(zero_shot_data, columns=["question", "answers", "input_final_prompts"])


zero_shot_path = os.path.join(saved_dir, f"llama_{task}_0shot.parquet")
few_shot_path = os.path.join(saved_dir, f"llama_{task}_fewshot.parquet")


df_zero.to_parquet(zero_shot_path, index=False)
df_few.to_parquet(few_shot_path, index=False)

print("Zero-shot parquet saved to:", zero_shot_path)
print("Few-shot parquet saved to:", few_shot_path)


Zero-shot parquet saved to: data/parquet/fixed/llama_musr_location_0shot.parquet
Few-shot parquet saved to: data/parquet/fixed/llama_musr_location_fewshot.parquet


In [85]:
print(df_few.iloc[0]['input_final_prompts'][0])

<|start_header_id|>user<|end_header_id|>
You are a helpful AI assistant that will answer reasoning questions. You may reason over the question but you will always say at the end "Answer: <Your Answer Letter Choice>". You must only pick one answer and you must end your response with "Answer: <Your Answer Letter Choice>" everytime!

Petra, the dedicated housewife, felt a thrill at the thought of her surprise anniversary dinner for her husband, Daniel. She had been skillfully maneuvering around Daniel's eagerness to pitch in without disappointing him or giving up her surprise. 

Daniel, ever-the-observant-husband, noted Petra's unusual enthusiasm about the day's menu. Despite not knowing the details, he appreciated her effort and wanted to help—silently, he decided to deploy his best skill—patiently awaiting his moment to help, maybe when Petra asked for something from the pantry. Amidst the excitement, there was Clara, their maid—ever diligent and efficient, trying to keep the surroundin